# Setup

## Import Packages

In [285]:
import os, json
import pandas as pd
from typing import Callable, Optional, Dict, Literal, List, Union
from pydantic import Field, root_validator

from langchain.agents.mrkl.base import ZeroShotAgent
from langchain.agents import AgentExecutor
from langchain.agents.agent_toolkits.base import BaseToolkit
from langchain.base_language import BaseLanguageModel
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, RetrievalQAWithSourcesChain
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain.tools.base import BaseTool
from langchain.utilities.google_serper import GoogleSerperAPIWrapper
from langchain.vectorstores import FAISS
from langchain.vectorstores.base import VectorStore

from utils import generate_agent_sequence
from exp.base import BaseExperiment

## Setup Directories

In [152]:
MAIN_DIR = ".."
EMB_DIR = os.path.join(MAIN_DIR, "data", "emb_store", "uc", "faiss", "text-embedding-ada-002", "v8-add-tables_2500_500")

with open(os.path.join(MAIN_DIR, "auth", "api_keys.json"), "r") as f:
    api_keys = json.load(f)
    
os.environ["OPENAI_API_KEY"] = api_keys["OPENAI_API_KEY"]

## Load Test Cases

In [153]:
with open(os.path.join(MAIN_DIR, "data", "queries", "uc_all.txt"),
          "r", encoding="utf-8-sig") as f:
    test_cases = f.readlines()
test_cases = [test_case.rstrip() for test_case in test_cases]

print("Total number of test cases:", len(test_cases))

Total number of test cases: 30


# ReAct and MRKL Agents

## Define Toolkit

In [208]:
def _print_func(text: str) -> None:
    print("\n")
    print(text)
    
def _get_input() -> str:
    print("Insert your text. Enter 'q' or press Ctrl-D (or Ctrl-Z on Windows) to end.")
    contents = []
    while True:
        try:
            line = input()
        except EOFError:
            break
        if line == "q":
            break
        contents.append(line)
    return "\n".join(contents)

class HumanTool(BaseTool):
    """Tool that asks user for input."""

    name = "Human"
    description = (
        "You can ask a human for additional information when you got stuck "
        "or you are not sure about the treatment recommendation. "
        "The input should be a question for the human to provide additional "
        "information about the patient, such as age, gender, whether the patient "
        "under treatment is a new patient or under maintenance, whether the patient "
        "has any special response/failure to any drugs or other conditions such as "
        "pregnancy, extraintestinale manifestations, etc"
    )
    prompt_func: Callable[[str], None] = Field(default_factory=lambda: _print_func)
    input_func: Callable = Field(default_factory=lambda: _get_input)

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the Human input tool."""
        self.prompt_func(query)
        return self.input_func()

class GeneralKnowledgeTool(BaseTool):
    name: str = "General Knowledge"
    description: str = "Useful for general knowledge question"
    llm: BaseLanguageModel
    
    @root_validator()
    def initiate_llm_chain(cls, values: Dict) -> Dict:
        values["llm_chain"] = LLMChain(
            prompt=ChatPromptTemplate.from_messages([HumanMessagePromptTemplate.from_template("Question: {question}")]),
            llm=values["llm"]
        )
        return values
    
    def _run(
        self, 
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None
        ) -> str:
        return self.llm_chain.run(query)
    
    def _arun(
        self,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None
        ) -> str:
        return NotImplementedError

class QASearchTool(BaseTool):
    name: str = "Docsearch QA Tool"
    description = (
        "Use this tool to search for document and answer questions related to "
        "treatment of ulcerative colitis from internal database"
    )
    llm: BaseLanguageModel
    docsearch: VectorStore
    prompt_template: Union[ChatPromptTemplate, PromptTemplate]
    chain_type: Literal["stuff", "map_reduce"] = "stuff"
    k: int = 4
    max_tokens_limit: int = 4500
    verbose: bool = True

    @root_validator()
    def generate_qa_chain(cls, values):
        values["chain"] = RetrievalQAWithSourcesChain.from_chain_type(
            llm=values["llm"],
            chain_type=values["chain_type"],
            retriever=values["docsearch"].as_retriever(search_kwargs={"k":values["k"]}),
            return_source_documents=True,
            chain_type_kwargs={"prompt": values["prompt_template"]},
            reduce_k_below_max_tokens=True,
            max_tokens_limit=values["max_tokens_limit"],
            verbose=values["verbose"]
            )
        return values

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None
        ) -> str:
        result = self.chain(query)
        return result["answer"]

    def _arun(
        self,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None
        ) -> str:
        return NotImplementedError

class GoogleSerperTool(BaseTool):
    """Tool that queries the Serper.dev Google search API."""

    name = "google_serper"
    description = (
        "Useful for when you need to search the internet for recommendations on patients profile "
        "which you don't know about. Input should be a search query."
    )
    serper_api_key: str
    
    @root_validator()
    def initiate_api(cls, values: Dict) -> Dict:
        values["api_wrapper"] = GoogleSerperAPIWrapper(serper_api_key=values["serper_api_key"])
        return values

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool."""
        return str(self.api_wrapper.run(query))

    async def _arun(
        self,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        return NotImplementedError
    
class HumanSearchRetrievalToolkit(BaseToolkit):
    serper_api_key: str
    llm: BaseLanguageModel
    qa_prompt: Union[ChatPromptTemplate, PromptTemplate]
    docsearch: VectorStore
    k: int = 4
    
    class Config:
        arbitrary_types_allowed = True
    
    def get_tools(self, tool_names: List = None) -> List[BaseTool]:
        all_tools_mapping = {
            "human": HumanTool(),
            "search": GoogleSerperTool(serper_api_key=self.serper_api_key),
            "retrieval_qa": QASearchTool(llm = self.llm, prompt_template = self.qa_prompt, docsearch = self.docsearch, k=self.k),
            "general": GeneralKnowledgeTool(llm=self.llm)
        }
        
        if not tool_names:
            return list(all_tools_mapping.values())
        
        tools = [all_tools_mapping[tool_name] for tool_name in tool_names]
        
        return tools

In [209]:
docsearch = FAISS.load_local(EMB_DIR, OpenAIEmbeddings())
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

system_prompt = """
Make reference to the context given to assess the scenario. If you do not know the answer. just say that "I don't know", don't try to make up an answer.
You are a physician assistant giving advice on treatment for moderate to severe ulcerative colitis (UC).

ANALYSE the given patient profile based on given query based on one of the following criteria:
- Whether treated patient is new patient or patient under maintenance
- Prior response to Infliximab
- Prior failure to Anti-TNF agents
- Prior failure to Vedolizumab
- Age
- Pregnancy
- Extraintestinale manifestations
- Pouchitis

FINALLY RETURN up to 2 TOP choices of biological drugs given patient profile. Explain the PROS and CONS of the 2 choices.

Remember if you are not sure about the answer, just say "I don't know", do not make up an answer!

{summaries}
"""

QA_PROMPT_TEMPLATE = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            system_prompt, input_variables=["summaries"]
        ),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
)

# Use All Tools

## Define Tools

In [210]:
toolkit = HumanSearchRetrievalToolkit(
    serper_api_key=api_keys["SERP_API"],
    llm = llm,
    qa_prompt = QA_PROMPT_TEMPLATE,
    docsearch=docsearch,
    k=5
)

tools = toolkit.get_tools(["retrieval_qa", "search", "general"])
for idx, tool in enumerate(tools):
    print(f"Tool number {idx+1}: {tool.name}")

Tool number 1: Docsearch QA Tool
Tool number 2: google_serper
Tool number 3: General Knowledge


## Define Agent Instructions

In [211]:
PREFIX = """You are a helpful and honest physician assistant giving advice on treatment for moderate to severe ulcerative colitis (UC).
If you do not know the answer, you will not make up an answer and instead ask further questions for clarifications

Your job is to ANALYSE the given patient profile based on given query based on one of the following criteria:
- Whether treated patient is new patient or patient under maintenance
- Prior response to Infliximab
- Prior failure to Anti-TNF agents
- Prior failure to Vedolizumab
- Age
- Pregnancy
- Extraintestinale manifestations
- Pouchitis

FINALLY RETURN up to 2 TOP choices of biological drugs given patient profile. Explain the PROS and CONS of the 2 choices.

You have access to the following tools:"""

FORMAT_INSTRUCTIONS = """Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]. Prioritize using Docsearch before using Internet search.
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question"""

SUFFIX = """Begin!

Patient Profile: {input}
Thought: {agent_scratchpad}"""

AGENT_PROMPT_1 = ZeroShotAgent.create_prompt(
    tools=tools,
    prefix=PREFIX,
    suffix=SUFFIX,
    format_instructions=FORMAT_INSTRUCTIONS,    
)

print(AGENT_PROMPT_1.template)

You are a helpful and honest physician assistant giving advice on treatment for moderate to severe ulcerative colitis (UC).
If you do not know the answer, you will not make up an answer and instead ask further questions for clarifications

Your job is to ANALYSE the given patient profile based on given query based on one of the following criteria:
- Whether treated patient is new patient or patient under maintenance
- Prior response to Infliximab
- Prior failure to Anti-TNF agents
- Prior failure to Vedolizumab
- Age
- Pregnancy
- Extraintestinale manifestations
- Pouchitis

FINALLY RETURN up to 2 TOP choices of biological drugs given patient profile. Explain the PROS and CONS of the 2 choices.

You have access to the following tools:

Docsearch QA Tool: Use this tool to search for document and answer questions related to treatment of ulcerative colitis from internal database
google_serper: Useful for when you need to search the internet for recommendations on patients profile which yo

## Define Agent Executor

In [223]:
agent_llm = LLMChain(
    llm = ChatOpenAI(model_name = "gpt-4", temperature = 0, max_tokens = 256),
    prompt = AGENT_PROMPT_1
)

REACT_AGENT_1 = ZeroShotAgent(llm_chain=agent_llm, allowed_tools = [tool.name for tool in tools])

agent_args = {
    "max_iterations": 5,
    "early_stopping_method": "generate",
    "return_intermediate_steps": True
    }
agent_executor = AgentExecutor.from_agent_and_tools(agent=REACT_AGENT_1, tools=tools, verbose=True, **agent_args)

## Run Test Case

In [197]:
results = [None] * 30

In [198]:
sample_case = test_cases[0]
results[0] = agent_executor(sample_case)



> Entering new AgentExecutor chain...
The patient is newly diagnosed with moderate ulcerative colitis and has extraintestinal manifestations in the form of joint issues. I need to consider the best biological drugs for this patient. I will use the Docsearch QA Tool to find the best treatment options for a newly diagnosed patient with these conditions.
Action: Docsearch QA Tool
Action Input: What are the best biological drugs for a newly diagnosed patient with moderate ulcerative colitis and articular extraintestinal manifestations?

> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.

Observation: Based on the data provided, Infliximab and Vedolizumab could be considered as the top choices for a newly diagnosed patient with moderate ulcerative colitis.

Infliximab was ranked highest for efficacy in biologic-naïve patients, meaning it has been shown to be very effective in inducing remission in patients who have not been previously treated with biological drugs. It 

In [224]:
sample_case = test_cases[1]
print("Patient Profile:", sample_case)
results[1] = agent_executor(sample_case)

Patient Profile: 70 year old female with newly diagnosed severe UC


> Entering new AgentExecutor chain...
The patient is newly diagnosed with severe ulcerative colitis and is of an older age. I need to consider the age factor and the severity of the disease while recommending biological drugs. I will use the Docsearch QA Tool to find the best treatment options for this patient profile.
Action: Docsearch QA Tool
Action Input: What are the recommended biological drugs for a newly diagnosed, severe ulcerative colitis in a 70 year old female patient? 

> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.

Observation: Based on the provided information, two top choices of biological drugs for a newly diagnosed, severe ulcerative colitis in a 70-year-old female patient could be Infliximab and Upadacitinib.

1. Infliximab: This drug targets the pro-inflammatory cytokine tumour necrosis factor-α (TNF-α), which has demonstrated efficacy in clinical trials in moderate to sever

In [226]:
sample_case = test_cases[14]
print("Patient Profile:", sample_case)
results[14] = agent_executor(sample_case)

Patient Profile: 55 year-old man with moderate to severe extensive ulcerative colitis who avlues convenience and limited time spent in hospital


> Entering new AgentExecutor chain...
The patient is middle-aged and has moderate to severe ulcerative colitis. He prefers convenience and limited hospital time. I need to consider these factors when recommending biological drugs. I will first use the Docsearch QA Tool to find suitable treatments for this patient profile.
Action: Docsearch QA Tool
Action Input: What are the recommended biological drugs for a 55 year-old man with moderate to severe extensive ulcerative colitis who values convenience and limited time spent in hospital?

> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.

Observation: Based on the information provided, two top choices of biological drugs for this patient could be Upadacitinib and Infliximab.

1. Upadacitinib: This drug ranked first in terms of clinical remission in all patients, in patients p

In [227]:
sample_case = test_cases[18]
print("Patient Profile:", sample_case)
results[18] = agent_executor(sample_case)

Patient Profile: 53 year-old man with moderate to severe extensive ulcerative colitis and low albumin levels


> Entering new AgentExecutor chain...
The patient is middle-aged and has moderate to severe ulcerative colitis. The low albumin levels indicate that the disease is active and the patient is losing protein. I need to consider the patient's age, disease severity, and albumin levels when recommending treatment. I will use the Docsearch QA Tool to find the best biological drugs for this patient profile.
Action: Docsearch QA Tool
Action Input: What are the best biological drugs for a 53 year-old man with moderate to severe extensive ulcerative colitis and low albumin levels? 

> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.

Observation: Based on the provided information, it's not clear whether the patient has been previously exposed to anti-TNF-α therapies or not. However, considering the general data from the studies, two top choices could be:

1. Upadaciti

In [228]:
sample_case = test_cases[19]
print("Patient Profile:", sample_case)
results[19] = agent_executor(sample_case)

Patient Profile: 42 year-old woman with severe ulcerative colitis and rare fistulating disease


> Entering new AgentExecutor chain...
The patient has severe ulcerative colitis and a rare fistulating disease. I need to consider the severity of the disease, the presence of the fistulating disease, and the patient's age and gender. I will first use the Docsearch QA Tool to find the best biological drugs for severe ulcerative colitis.
Action: Docsearch QA Tool
Action Input: What are the best biological drugs for severe ulcerative colitis?

> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.

Observation: Based on the provided information, two top choices of biological drugs for moderate to severe ulcerative colitis could be Infliximab and Upadacitinib.

1. Infliximab: 
   - Pros: Infliximab has demonstrated efficacy in clinical trials for moderate to severe UC. It targets the pro-inflammatory cytokine tumour necrosis factor-α (TNF-α), which plays a significant role in t

In [229]:
sample_case = test_cases[20]
print("Patient Profile:", sample_case)
results[20] = agent_executor(sample_case)

Patient Profile: 68 year-old man with extensive moderate to severe ulcerative colitis who has prostate cancer which has been treated 5 years ago.


> Entering new AgentExecutor chain...
The patient is elderly and has a history of prostate cancer. His ulcerative colitis is extensive and moderate to severe. I need to consider his age and cancer history when recommending biological drugs. I will use the Docsearch QA Tool to find suitable treatments for this patient profile.
Action: Docsearch QA Tool
Action Input: Treatment for moderate to severe ulcerative colitis in elderly patients with history of cancer

> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.

Observation: I'm sorry, but the documents provided do not contain specific information about the treatment of moderate to severe ulcerative colitis in elderly patients with a history of cancer.
Thought:The documents did not provide specific information for this patient's profile. I will now use the google_serper to

In [242]:
results[20]["intermediate_steps"][0][1]

"I'm sorry, but the documents provided do not contain specific information about the treatment of moderate to severe ulcerative colitis in elderly patients with a history of cancer."

In [246]:
other_case_idx = [13, 15, 16, 17, 21, 22, 23, 24, 25, 26, 27, 28, 29]

for case_idx in other_case_idx:
    test_case = test_cases[case_idx]
    print(f"Case {case_idx}: Patient Profile: {test_case}")
    results[case_idx] = agent_executor(test_case)

Patient Profile: 44 year-old woman with moderate to severe extensive ulcerative colitis and rheumatoid arthritis.


> Entering new AgentExecutor chain...
The patient has moderate to severe ulcerative colitis and rheumatoid arthritis. I need to consider the patient's age, the severity of her conditions, and the fact that she has two autoimmune diseases. I should look for biological drugs that can treat both conditions.
Action: Docsearch QA Tool
Action Input: What are the recommended biological drugs for treating moderate to severe ulcerative colitis and rheumatoid arthritis? 

> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.

Observation: I don't know
Thought:I didn't get the information I needed from the Docsearch QA Tool. I should try using the internet search to find the recommended biological drugs for treating moderate to severe ulcerative colitis and rheumatoid arthritis.
Action: google_serper
Action Input: Recommended biological drugs for treating moderate t

## Write Output

### Json

In [279]:
json_output = []

for result in results:
    json_output.append(
        {
            "question": result["input"],
            "thought_sequence": generate_agent_sequence(result),
            "answer": result["output"]
        }
    )

exp_folder = os.path.join(MAIN_DIR, "artifacts", "gpt-4-agent-exp-1")
if not os.path.exists(exp_folder):
    os.makedirs(exp_folder, exist_ok=True)

with open(os.path.join(MAIN_DIR, "artifacts", "gpt-4-agent-exp-1", "result.json"), "w") as f:
    json.dump(json_output, f)

## CSV

In [287]:
BaseExperiment.convert_prompt_to_string(
    AGENT_PROMPT_1
)

"You are a helpful and honest physician assistant giving advice on treatment for moderate to severe ulcerative colitis (UC).\nIf you do not know the answer, you will not make up an answer and instead ask further questions for clarifications\n\nYour job is to ANALYSE the given patient profile based on given query based on one of the following criteria:\n- Whether treated patient is new patient or patient under maintenance\n- Prior response to Infliximab\n- Prior failure to Anti-TNF agents\n- Prior failure to Vedolizumab\n- Age\n- Pregnancy\n- Extraintestinale manifestations\n- Pouchitis\n\nFINALLY RETURN up to 2 TOP choices of biological drugs given patient profile. Explain the PROS and CONS of the 2 choices.\n\nYou have access to the following tools:\n\nDocsearch QA Tool: Use this tool to search for document and answer questions related to treatment of ulcerative colitis from internal database\ngoogle_serper: Useful for when you need to search the internet for recommendations on patien

In [310]:
GROUNDTRUTH_PATH = os.path.join(MAIN_DIR, "data", "queries", "uc_all_gt.csv")

gt = pd.read_csv(GROUNDTRUTH_PATH, encoding="ISO-8859-1")

info = {"question": [result["input"] for result in results]}

if gt is not None:
    info["gt_rec1"] = gt["Recommendation 1"].tolist()
    info["gt_rec2"] = gt["Recommendation 2"].tolist()
    info["gt_rec3"] = gt["Recommendation 3"].tolist()
    info["gt_avoid"] = gt["Drug Avoid"].tolist()
    info["gt_reason"] = gt["Reasoning"].tolist()
    
info["agent_prompt"] = [
    BaseExperiment.convert_prompt_to_string(
        AGENT_PROMPT_1
    )
] * len(info["question"])

info["retrieval_qa_prompt"] = [
    BaseExperiment.convert_prompt_to_string(
        QA_PROMPT_TEMPLATE
    )
] * len(info["question"])

info["raw_answer"] = [result["output"] for result in results]
info["answer1"] = [None] * len(info["question"])
info["answer2"] = [None] * len(info["question"])

info["thought_sequence"] = [generate_agent_sequence(result) for result in results]

tool_names = [tool.name for tool in tools]
for tool in tool_names:
    usage = ["YES" if tool in thought_seq else "NO" for thought_seq in info["thought_sequence"]]
    info[tool] = usage

panda_df = pd.DataFrame(info)

In [312]:
output_csv = os.path.join(MAIN_DIR, "artifacts", "gpt-4-agent-exp-1", "result.csv")
panda_df.to_csv(output_csv, header=True)